In [9]:
import torch
import torch.nn as nn
import torchvision 
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader


In [ ]:
'''
Formula of define output size after convolution:

(W-F+2P)/S + 1

where, W = width of input
P = Padding
S = Stride
F = Filter size
'''

In [11]:
# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define hyper parameters
num_epochs = 4
batch_size = 4
learning_rate = 0.001


# define transformations
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ]
)


# Import dataset
train_dataset = torchvision.datasets.CIFAR10(root=r'C:\Pytorch Practice\Data\cifar10_dataset', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = torchvision.datasets.CIFAR10(root=r'C:\Pytorch Practice\Data\cifar10_dataset', train=False, transform=transforms.ToTensor())
# Dataloaders
train_loaders =  DataLoader(dataset= train_dataset,batch_size=batch_size,shuffle= True)
test_loaders =  DataLoader(dataset= test_dataset,batch_size=batch_size,shuffle= False)

# Define classes
classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')


Files already downloaded and verified


In [12]:

# Convolutional Neural Network
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,6,5) 
        self.pool = nn.MaxPool2d(2,2)   
        self.conv2 = nn.Conv2d(6,16,5)  
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x))) # [3,32,32]-> [5,28,28]->[5,14,14]
        x = self.pool(F.relu(self.conv2(x))) # [5,14,14]-> [16,10,10]->[16,5,5]
        x = x.view(-1,16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x                                       
    
    
# Define model
model = ConvNet().to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training 
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loaders) :
        # Move data to device
        images = images.to(device)
        labels = labels.to(device)
        # Forward pass
        
        outputs = model(images)
        loss = criterion(outputs,labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        if (i+1)%2000==0:
            print(f'Epoch {epoch+1}/{num_epochs}, Loss = {loss.item():.4f}')
        


Epoch 1/4, Loss = 2.3306
Epoch 1/4, Loss = 2.2960
Epoch 1/4, Loss = 2.3034
Epoch 1/4, Loss = 2.2939
Epoch 1/4, Loss = 2.2724
Epoch 1/4, Loss = 2.2466
Epoch 2/4, Loss = 2.3276
Epoch 2/4, Loss = 2.2802
Epoch 2/4, Loss = 2.1187
Epoch 2/4, Loss = 2.3018
Epoch 2/4, Loss = 2.4412
Epoch 2/4, Loss = 1.8174
Epoch 3/4, Loss = 2.5495
Epoch 3/4, Loss = 2.1183
Epoch 3/4, Loss = 1.8137
Epoch 3/4, Loss = 2.5006
Epoch 3/4, Loss = 1.7414
Epoch 3/4, Loss = 1.8193
Epoch 4/4, Loss = 1.2425
Epoch 4/4, Loss = 1.9690
Epoch 4/4, Loss = 0.9820
Epoch 4/4, Loss = 2.4040
Epoch 4/4, Loss = 1.9499
Epoch 4/4, Loss = 2.3643


In [13]:
# Check data
data = iter(train_loaders)
images, labels= next(data)

images.shape

torch.Size([4, 3, 32, 32])